<a href="https://colab.research.google.com/github/sharmadeep441/Apple-Shelf-Life-Prediction-System/blob/main/Apple_Shelf_life_prection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob 
import cv2
import os
import seaborn as sns
import pandas as pd
 

In [ ]:
#Resize images to
SIZE = 128

Linking ntbk with my dataset and adding labels.

In [ ]:
train_images = []
train_labels = [] 

for directory_path in glob.glob("/content/drive/MyDrive/AppleDataset/Training/*"):
    label = directory_path.split("\\")[-1] #labels as per subclasses i.e shelf life
    print(label)

    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")): #adding images name
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR) #Reading color images
        img = cv2.resize(img, (SIZE, SIZE)) #Resize images
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)


/content/drive/MyDrive/AppleDataset/Training/first_5
/content/drive/MyDrive/AppleDataset/Training/first_5/frame344.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame421.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame445.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame352.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame353.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame343.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame348.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame356.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame347.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame455.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame369.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame354.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame401.jpg
/content/drive/MyDrive/AppleDataset/Training/first_5/frame405.jpg
/content/drive/MyDrive/

Converting images into arrays

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)


In [ ]:
train_labels

array(['/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/first_5',
       '/content/drive/MyDrive/AppleDataset/Training/fi

In [ ]:
test_images = []
test_labels = [] 
for directory_path in glob.glob("/content/drive/MyDrive/AppleDataset/Testing/*"):
    test_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #Optional
        test_images.append(img)
        test_labels.append(test_label)

In [ ]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
test_labels

array(['/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/MyDrive/AppleDataset/Testing/next_5',
       '/content/drive/M

Encoding

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels) 

test_labels_encoded.shape

(48,)

In [ ]:
train_images.shape

(72, 128, 128, 3)

Distributing in training and test data

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
#If you only have one dataset then split here
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

x_train.shape

(72, 128, 128, 3)

In [ ]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

Feature Extractor

In [ ]:
def feature_extractor(dataset):
    x_train = dataset
    image_dataset = pd.DataFrame()
    for image in range(x_train.shape[0]):  #iterate through each file 
        #print(image)
        
        df = pd.DataFrame()  #Temporary data frame to capture information for each loop.
        #Reset dataframe to blank after each loop.
        
        input_img = x_train[image, :,:,:]
        img = input_img
    ################################################################
    #START ADDING DATA TO THE DATAFRAME
    #Add feature extractors, e.g. edge detection, smoothing, etc. 
            
         # FEATURE 1 - Pixel values
         
        #Add pixel values to the data frame
        pixel_values = img.reshape(-1)
        df['Pixel_Value'] = pixel_values   #Pixel value itself as a feature
        #df['Image_Name'] = image   #Capture image name as we read multiple images
        
        # FEATURE 2 - Bunch of Gabor filter responses
        
                #Generate Gabor features
        num = 1  #To count numbers up in order to give Gabor features a lable in the data frame
        kernels = []
        for theta in range(2):   #Define number of thetas
            theta = theta / 4. * np.pi
            for sigma in (1, 3):  #Sigma with 1 and 3
                lamda = np.pi/4
                gamma = 0.5
                gabor_label = 'Gabor' + str(num)  #Label Gabor columns as Gabor1, Gabor2, etc.
    #                print(gabor_label)
                ksize=9
                kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                kernels.append(kernel)
                #Now filter the image and add values to a new column 
                fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
                print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                num += 1  #Increment for gabor column label
                
         
       
        image_dataset = image_dataset.append(df)
        
    return image_dataset

In [ ]:
#Extract features from training images
image_features = feature_extractor(x_train)

Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor2 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor3 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor4 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor2 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor3 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor4 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor2 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor3 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor4 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853

In [ ]:
image_features.shape

(3538944, 5)

In [ ]:
#Reshape to a vector for Random Forest 
n_features = image_features.shape[1]
image_features = np.expand_dims(image_features, axis=0)
X_for_RF = np.reshape(image_features, (x_train.shape[0], -1))  #Reshape to #images, features

In [ ]:
X_for_RF.shape

(72, 245760)

Regression Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_for_RF, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [ ]:
#Predict on Test data
#Extract features from test data and reshape, just like training data
test_features = feature_extractor(x_test)
test_features = np.expand_dims(test_features, axis=0)
test_for_RF = np.reshape(test_features, (x_test.shape[0], -1))

test_for_RF.shape

Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor2 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor3 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor4 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor2 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor3 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor4 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor2 : theta= 0.0 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor3 : theta= 0.7853981633974483 : sigma= 1 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor4 : theta= 0.7853981633974483 : sigma= 3 : lamda= 0.7853981633974483 : gamma= 0.5
Gabor1 : theta= 0.0 : sigma= 1 : lamda= 0.7853

(48, 245760)

In [ ]:
#Predict on test
test_prediction = regressor.predict(test_for_RF)
#Inverse le transform to get original label back. 
# test_prediction = le.inverse_transform(test_prediction)
test_prediction

array([0.7, 0.7, 0.7, 0.8, 0.8, 0.2, 0.8, 0.7, 0.7, 0.7, 0.7, 0.8, 0.7,
       0. , 0.7, 0.7, 0.7, 0.8, 0.6, 0.2, 0. , 0.6, 0.7, 0.8, 0. , 0. ,
       0.2, 0. , 0. , 0. , 0. , 0.2, 0. , 0. , 0. , 0.2, 0. , 0. , 0. ,
       0. , 0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. ])

In [ ]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

Checking Accuracy

In [ ]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, test_prediction), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, test_prediction), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, test_prediction), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, test_prediction), 2)) 
print("R2 score =", round(sm.r2_score(y_test, test_prediction), 2))

Mean absolute error = 0.21
Mean squared error = 0.11
Median absolute error = 0.2
Explain variance score = 0.7
R2 score = 0.58
